In [1]:
!pip install -q xlrd
!git clone https://github.com/Lossophy/BINA-Projekt.git
!pip install panel --quiet
!pip install jupyter_bokeh

Cloning into 'BINA-Projekt'...
remote: Enumerating objects: 300, done.
remote: Counting objects: 100% (135/135), done.
remote: Compressing objects: 100% (132/132), done.
remote: Total 300 (delta 85), reused 8 (delta 3), pack-reused 165 (from 1)
Receiving objects: 100% (300/300), 5.26 MiB | 16.36 MiB/s, done.
Resolving deltas: 100% (174/174), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 59.4 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Succes

In [2]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
#import ipywidgets as widgets
import time
from IPython.display import display, clear_output
import panel as pn
pn.extension('plotly',sizing_mode="stretch_width")

# Harmlose Bokeh-Warnmeldungen ausblenden
import warnings
from bokeh.util.warnings import BokehUserWarning
warnings.filterwarnings("ignore", category=BokehUserWarning)

### F3: Wie hat sich der Anteil erneuerbarer Energien aus inländischer Produktion gegenüber Importen fossiler Energieträger über die Jahre verändert?

Methodischer Ansatz

Zur Beantwortung der Fragestellung, wie sich der Anteil erneuerbarer Energien aus inländischer Produktion im Vergleich zu den Importen fossiler Energieträger über die Jahre entwickelt hat, wurde die Schweizerische Gesamtenergiestatistik für den Zeitraum von 1980 bis 2023 analysiert. Dieser methodische Ansatz orientiert sich an Schritt 3 "Daten analysieren" der CPA-Guideline "From Data to Decisions". Für die inländische Produktion erneuerbarer Energien wurden die Energieträger "Holzenergie", "Wasserkraft" und "Uebrige erneuerbare Energien" berücksichtigt. Zusätzlich wurde, basierend auf den Erkenntnissen der Schweizerischen Gesamtenergiestatistik 2023 und der gängigen Praxis, wonach der biogene Anteil von Abfällen einen signifikanten erneuerbaren Beitrag leistet, der Energieträger "Müll und Industrieabfälle" zu 50% seiner Inlandproduktion den erneuerbaren Energien zugerechnet. Diese konsistente Annahme ermöglicht eine umfassendere Betrachtung des inländischen erneuerbaren Potenzials. Demgegenüber stehen die Importe fossiler Energieträger, welche "Erdölprodukte", "Gas", "Kohle" und "Rohöl" umfassen.

Die jährlichen Energiemengen (in Terajoule, TJ) wurden für beide Hauptkategorien aggregiert. Darauf aufbauend erfolgte eine detaillierte Analyse der Entwicklung der absoluten Mengen, der relativen Anteile an der Summe beider Kategorien sowie des direkten Verhältnisses der inländischen erneuerbaren Produktion zu den fossilen Importen. Die Visualisierung dieser Entwicklungen erfolgte mittels Linien- und Flächendiagrammen, um Trends und Veränderungen über den betrachteten Zeitraum transparent darzustellen.

In [3]:
# Datei einlesen
df = pd.read_csv('/content/BINA-Projekt/Data/ogd115_gest_bilanz.csv')

# Daten überprüfen
print(df.head())

   Jahr           Rubrik Energietraeger        TJ
0  1980  Bruttoverbrauch   Elektrizität  -29450.0
1  1980  Bruttoverbrauch  Erdölprodukte  321250.0
2  1980  Bruttoverbrauch      Fernwärme       0.0
3  1980  Bruttoverbrauch            Gas   36280.0
4  1980  Bruttoverbrauch    Holzenergie   26280.0


In [4]:
# Grundlegende Datenaufbereitung
df['TJ'] = pd.to_numeric(df['TJ'], errors='coerce').fillna(0)
df['Jahr'] = df['Jahr'].astype(int)

# --- Definitionen für F3 ---
erneuerbare_inland_basis_f3 = ["Holzenergie", "Wasserkraft", "Uebrige erneuerbare Energien"]
muell_energietraeger_f3 = "Müll und Industrieabfälle" # Einzelner String für leichtere Prüfung
# erneuerbare_inland_mit_muell_f3 wird jetzt in prepare_f3_data dynamischer gehandhabt
fossile_import_traeger_f3 = ["Erdölprodukte", "Gas", "Kohle", "Rohöl"]

# Farbcodierung für F3
farben_erneuerbar_inland_f3 = {
    "Holzenergie": "#1f77b4", "Wasserkraft": "#aec7e8", "Uebrige erneuerbare Energien": "#72bcd4",
    "Müll und Industrieabfälle (50% erneuerbar)": "#2ca02c" # Angepasster Name für Legende
}
farben_fossil_import_f3 = {
    "Erdölprodukte": "#ff7f0e", "Gas": "#ffbb78", "Kohle": "#d62728", "Rohöl": "#ff9896"
}

# color_map_f3_szenario1_detail bleibt wie es ist
color_map_f3_szenario1_detail = {
    **{k: v for k, v in farben_erneuerbar_inland_f3.items() if k != "Müll und Industrieabfälle (50% erneuerbar)"},
    **farben_fossil_import_f3
}

# color_map_f3_szenario2_detail wird angepasst, um den neuen Namen für Müll zu verwenden
color_map_f3_szenario2_detail = {**farben_erneuerbar_inland_f3, **farben_fossil_import_f3}


# --- Hilfsfunktion zur Datenaggregation (angepasst für 50% Müll) ---
def prepare_f3_data(df_input, erneuerbare_liste_basis, fossile_liste, muell_anteil_prozent=0, szenario_label=""):
    df_erneuerbare_inland = df_input[
        (df_input['Rubrik'] == "Inlandproduktion") &
        (df_input['Energietraeger'].isin(erneuerbare_liste_basis))
    ].copy() # .copy() um SettingWithCopyWarning zu vermeiden

    # Behandlung von Müll und Industrieabfälle
    df_muell_inland = pd.DataFrame() # Initialisieren für den Fall, dass muell_anteil_prozent = 0
    if muell_anteil_prozent > 0:
        df_muell_temp = df_input[
            (df_input['Rubrik'] == "Inlandproduktion") &
            (df_input['Energietraeger'] == muell_energietraeger_f3)
        ].copy()
        df_muell_temp['TJ'] = df_muell_temp['TJ'] * (muell_anteil_prozent / 100.0)
        df_muell_temp['Energietraeger'] = f"{muell_energietraeger_f3} ({muell_anteil_prozent}% erneuerbar)" # Für Legende
        df_muell_inland = df_muell_temp

    # Kombinieren der Basis-Erneuerbaren mit dem Müll-Anteil
    df_ern_inland_detail_final = pd.concat([df_erneuerbare_inland, df_muell_inland])

    # Detail-Daten für Plotting vorbereiten
    df_ern_inland_detail_grouped = df_ern_inland_detail_final.groupby(['Jahr', 'Energietraeger'])['TJ'].sum().reset_index()
    df_ern_inland_detail_grouped['Hauptkategorie'] = f'Inländ. Erneuerbare {szenario_label}'.strip()


    ern_inland_summe = df_ern_inland_detail_final.groupby('Jahr')['TJ'].sum().reset_index()
    ern_inland_summe = ern_inland_summe.rename(columns={'TJ': 'Erneuerbare_Inland_TJ'})

    df_foss_import_detail = df_input[
        (df_input['Rubrik'] == "Import") &
        (df_input['Energietraeger'].isin(fossile_liste))
    ].groupby(['Jahr', 'Energietraeger'])['TJ'].sum().reset_index()
    df_foss_import_detail['Hauptkategorie'] = 'Importe fossiler Energieträger'

    foss_import_summe = df_foss_import_detail.groupby('Jahr')['TJ'].sum().reset_index()
    foss_import_summe = foss_import_summe.rename(columns={'TJ': 'Fossile_Import_TJ'})

    df_detail_kombiniert_final = pd.concat([df_ern_inland_detail_grouped, df_foss_import_detail])

    df_vergleich = pd.merge(ern_inland_summe, foss_import_summe, on='Jahr', how='outer').fillna(0)
    df_vergleich = df_vergleich.sort_values(by='Jahr').reset_index(drop=True)

    df_vergleich['Verhaeltnis_ErnIn_vs_FossImp'] = df_vergleich['Erneuerbare_Inland_TJ'] / df_vergleich['Fossile_Import_TJ'].replace(0, np.nan)
    df_vergleich['Verhaeltnis_ErnIn_vs_FossImp'] = df_vergleich['Verhaeltnis_ErnIn_vs_FossImp'].fillna(0)

    summe_energien = df_vergleich['Erneuerbare_Inland_TJ'] + df_vergleich['Fossile_Import_TJ']
    label_suffix = f" {szenario_label}".rstrip()
    df_vergleich[f'Anteil_Erneuerbare_Inland_Prozent{label_suffix}'] = (df_vergleich['Erneuerbare_Inland_TJ'] / summe_energien.replace(0, np.nan) * 100).fillna(0)
    df_vergleich[f'Anteil_Fossile_Import_Prozent{label_suffix}'] = (df_vergleich['Fossile_Import_TJ'] / summe_energien.replace(0, np.nan) * 100).fillna(0)

    return df_detail_kombiniert_final, df_vergleich

# --- Daten für Szenarien vorbereiten ---
# Szenario 1: Basis Erneuerbare (ohne Müll)
df_detail_s1_f3, df_vergleich_s1_f3 = prepare_f3_data(df, erneuerbare_inland_basis_f3, fossile_import_traeger_f3, muell_anteil_prozent=0, szenario_label="(Basis)")

# Szenario 2: Erneuerbare INKLUSIVE 50% von Müll und Industrieabfälle
df_detail_s2_f3, df_vergleich_s2_f3 = prepare_f3_data(df, erneuerbare_inland_basis_f3, fossile_import_traeger_f3, muell_anteil_prozent=50, szenario_label="(inkl. 50% Müll)")

In [5]:
def plot_f3_s2_absolut_aggregiert_tab():
    if df_vergleich_s2_f3.empty: return go.Figure().update_layout(title_text="Daten für Szenario 2 nicht geladen") # Geändert auf df_vergleich_s2_f3
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df_vergleich_s2_f3['Jahr'], y=df_vergleich_s2_f3['Erneuerbare_Inland_TJ'],
                             mode='lines+markers', name='Inländ. Erneuerbare (inkl. 50% Müll)', line=dict(color=farben_erneuerbar_inland_f3["Müll und Industrieabfälle (50% erneuerbar)"]))) # Legendenname und Farbe angepasst
    fig.add_trace(go.Scatter(x=df_vergleich_s2_f3['Jahr'], y=df_vergleich_s2_f3['Fossile_Import_TJ'],
                             mode='lines+markers', name='Importe fossiler Energieträger', line=dict(color=farben_fossil_import_f3["Erdölprodukte"])))

    fig.update_layout(
        title_text="Abs. Mengen (aggregiert): Erneuerbare (inkl. 50% Müll) vs. Fossile Importe", # Titel angepasst
        title_x=0.5,
        xaxis_title="Jahr",
        yaxis_title="Energiemenge (TJ)",
        hovermode="x unified",
        legend_title_text='Kategorie',
        legend=dict(traceorder="reversed"))
    return fig

beschreibung_s2_absolut_aggregiert = """
**Grafik: Absolute Mengen (aggregiert): Inl. Erneuerbare vs. Fossile Importe (Szen. Müll)**

Diese Grafik zeigt die absolute Entwicklung der Energiemengen (in Terajoule, TJ) über die Jahre.
Verglichen werden die Summe der inländisch produzierten erneuerbaren Energien mit der Summe der importierten fossilen Energieträger.
"""

In [6]:
def plot_f3_s2_absolut_detail_tab():
    if df_detail_s2_f3.empty: return go.Figure().update_layout(title_text="Daten für Szenario 2 nicht geladen") # Geändert auf df_detail_s2_f3
    # Erstelle eine benutzerdefinierte Sortierreihenfolge für die Legende und Stapelung
    kategorie_reihenfolge_detail_s2 = [et for et in erneuerbare_inland_basis_f3]
    kategorie_reihenfolge_detail_s2.append("Müll und Industrieabfälle (50% erneuerbar)") # Angepasster Name
    kategorie_reihenfolge_detail_s2.extend(fossile_import_traeger_f3)

    fig = px.area(df_detail_s2_f3, x="Jahr", y="TJ", color="Energietraeger", line_group="Hauptkategorie", # line_group beibehalten oder anpassen, falls nötig
                  color_discrete_map=color_map_f3_szenario2_detail,
                  title="Abs. Mengen (detailliert): Erneuerbare (inkl. 50% Müll) vs. Fossile Importe", # Titel angepasst
                  labels={"TJ": "Energiemenge (TJ)", "Energietraeger": "Energieträger"},
                  category_orders={"Energietraeger": kategorie_reihenfolge_detail_s2})

    fig.update_layout(
        title_x=0.5,
        legend_title_text='Energieträger',
        hovermode="x unified",
        legend=dict(traceorder="reversed"))
    return fig

beschreibung_s2_absolut_detail = """
**Grafik: Absolute Mengen (detailliert): Inl. Erneuerbare vs. Fossile Importe (Szen. Müll)**

Dieses Flächendiagramm stellt die absoluten Energiemengen (in Terajoule, TJ) der einzelnen Energieträger über die Jahre dar.
Die Flächen sind gestapelt, wobei die obere Gruppe die inländischen erneuerbaren Energien und die untere Gruppe die importierten fossilen Energieträger detailliert aufschlüsselt.
"""

In [7]:
def plot_f3_s2_anteil_100prozent_aggregiert_tab():
    if df_vergleich_s2_f3.empty: return go.Figure().update_layout(title_text="Daten für Szenario 2 nicht geladen")
    df_anteile_melted = df_vergleich_s2_f3.melt(id_vars=['Jahr'],
                                              value_vars=['Anteil_Erneuerbare_Inland_Prozent (inkl. 50% Müll)', 'Anteil_Fossile_Import_Prozent (inkl. 50% Müll)'], # Angepasste Spaltennamen
                                              var_name='Kategorie_Anteil', value_name='Prozentualer_Anteil')
    df_anteile_melted['Kategorie_Anteil'] = df_anteile_melted['Kategorie_Anteil'].replace({
        'Anteil_Erneuerbare_Inland_Prozent (inkl. 50% Müll)': 'Anteil Inländ. Erneuerbare (inkl. 50% Müll)', # Angepasst
        'Anteil_Fossile_Import_Prozent (inkl. 50% Müll)': 'Anteil Fossile Importe'
    })
    fig = px.area(df_anteile_melted, x="Jahr", y="Prozentualer_Anteil", color="Kategorie_Anteil",
                  title="Anteilige Entwicklung (aggregiert, 100% Ansicht, inkl. 50% Müll)", # Titel angepasst
                  labels={"Prozentualer_Anteil": "Prozentualer Anteil (%)"},
                  color_discrete_map={'Anteil Inländ. Erneuerbare (inkl. 50% Müll)': farben_erneuerbar_inland_f3["Müll und Industrieabfälle (50% erneuerbar)"], 'Anteil Fossile Importe': '#ff7f0e'}, # Farbe angepasst
                  groupnorm='percent')

    fig.update_layout(
        title_x=0.5,
        legend_title_text='Energiekategorie',
        hovermode="x unified",
        legend=dict(traceorder="reversed"))
    fig.update_yaxes(ticksuffix="%")
    return fig

beschreibung_s2_anteil_100_aggregiert = """
**Grafik: Anteilige Entwicklung (aggregiert, 100% Ansicht, Szen. Müll)**

Dieses gestapelte Flächendiagramm zeigt die prozentuale Verteilung zwischen der Summe der inländischen erneuerbaren Energien und der Summe der importierten fossilen Energieträger über die Jahre.
Die Y-Achse summiert sich für jedes Jahr auf 100%, wodurch die relativen Anteile der beiden Hauptkategorien direkt vergleichbar werden.
"""

In [8]:
def plot_f3_s2_verhaeltnis_tab():
    if df_vergleich_s2_f3.empty: return go.Figure().update_layout(title_text="Daten für Szenario 2 nicht geladen") # Geändert auf df_vergleich_s2_f3
    fig = px.line(df_vergleich_s2_f3, x="Jahr", y="Verhaeltnis_ErnIn_vs_FossImp",
                  title="Verhältnis Inl. Erneuerbare (inkl. 50% Müll) zu Fossilen Importen", # Titel angepasst
                  labels={"Verhaeltnis_ErnIn_vs_FossImp": "Verhältnis"}, markers=True,
                  color_discrete_sequence=[farben_erneuerbar_inland_f3["Müll und Industrieabfälle (50% erneuerbar)"]]) # Farbe hinzugefügt
    fig.add_hline(y=1, line_dash="dash", line_color="grey", annotation_text="Gleichstand")

    fig.update_layout(
        title_x=0.5,
        hovermode="x unified",
        legend=dict(traceorder="reversed"))
    return fig

beschreibung_s2_verhaeltnis = """
**Grafik: Verhältnis Inl. Erneuerbare zu Fossilen Importen (Szen. Müll)**

Dieses Liniendiagramm stellt das Verhältnis der Energiemenge aus inländischer erneuerbarer Produktion zu der Menge der importierten fossilen Energieträger dar.
Ein Wert über 1 (gestrichelte Linie) bedeutet, dass mehr erneuerbare Energie im Inland produziert als fossile Energie importiert wurde; ein Wert unter 1 das Gegenteil.
"""

In [9]:
def plot_f3_s2_anteil_summe_tab():
    if df_vergleich_s2_f3.empty:
        return go.Figure().update_layout(title_text="Daten für Szenario 2 nicht geladen")

    fig = px.line(df_vergleich_s2_f3,
                  x='Jahr',
                  y='Anteil_Erneuerbare_Inland_Prozent (inkl. 50% Müll)', # Angepasster Spaltenname
                  title='Anteil Inl. Erneuerbare (inkl. 50% Müll) an (Inl. Erneuerbare (inkl. 50% Müll) + Fossile Importe)', # Titel angepasst
                  labels={'Anteil_Erneuerbare_Inland_Prozent (inkl. 50% Müll)': 'Anteil (%)'}, # Label angepasst
                  markers=True,
                  line_shape='linear',
                  color_discrete_sequence=[farben_erneuerbar_inland_f3["Müll und Industrieabfälle (50% erneuerbar)"]]) # Passende Farbe

    fig.update_layout(
        title_x=0.5,
        xaxis_title="Jahr",
        yaxis_title="Anteil (%)",
        hovermode="x unified",
        yaxis_ticksuffix="%",
        legend=dict(traceorder="reversed")
    )
    return fig

beschreibung_s2_anteil_summe = """
**Grafik: Anteil Inl. Erneuerbare an Summe (Inl. Ern. + Foss. Imp., Szen. Müll)**

Dieses Liniendiagramm zeigt den prozentualen Anteil, den die inländisch produzierten erneuerbaren Energien an der Gesamtmenge aus (inländischen Erneuerbaren + fossilen Importen) über die Jahre ausmachen.
Die Y-Achse gibt diesen Anteil in Prozent an.
"""

In [10]:
def plot_f3_s2_anteil_100prozent_detail_tab():
    if df_detail_s2_f3.empty:
        return go.Figure().update_layout(title_text="Daten für Szenario 2 (inkl. 50% Müll) nicht geladen") # Titel angepasst

    df_detail_s2_f3_sorted = df_detail_s2_f3.copy()
    # Erstelle eine benutzerdefinierte Sortierreihenfolge für die Legende und Stapelung
    kategorie_reihenfolge_detail_s2 = [et for et in erneuerbare_inland_basis_f3]
    kategorie_reihenfolge_detail_s2.append("Müll und Industrieabfälle (50% erneuerbar)") # Angepasster Name
    kategorie_reihenfolge_detail_s2.extend(fossile_import_traeger_f3)

    df_detail_s2_f3_sorted['Energietraeger'] = pd.Categorical(
        df_detail_s2_f3_sorted['Energietraeger'],
        categories=kategorie_reihenfolge_detail_s2,
        ordered=True
    )
    df_detail_s2_f3_sorted = df_detail_s2_f3_sorted.sort_values(by=['Jahr', 'Energietraeger'])

    fig = px.area(df_detail_s2_f3_sorted,
                  x="Jahr",
                  y="TJ",
                  color="Energietraeger",
                  line_group="Energietraeger",
                  groupnorm='percent',
                  title="Anteilige Entwicklung (detailliert, 100% Ansicht): Erneuerbare (inkl. 50% Müll) vs. Fossile Importe", # Titel angepasst
                  labels={"TJ": "Prozentualer Anteil (%)", "Energietraeger": "Energieträger"},
                  color_discrete_map=color_map_f3_szenario2_detail, # Verwendet die aktualisierte color_map
                  category_orders={"Energietraeger": kategorie_reihenfolge_detail_s2}
                 )

    fig.update_layout(
        title_x=0.5,
        legend_title_text='Energieträger',
        hovermode="x unified",
        yaxis_ticksuffix="%",
        legend=dict(traceorder="reversed")
    )
    return fig

beschreibung_s2_anteil_100_detail = """
**Grafik: Anteilige Entwicklung (detailliert, 100% Ansicht, Szen. Müll)**

Dieses gestapelte Flächendiagramm visualisiert die prozentuale Zusammensetzung der betrachteten Energiemixe (Inländische Erneuerbare und Importe fossiler Energieträger) über die Jahre, aufgeschlüsselt nach einzelnen Energieträgern.
Die Y-Achse summiert sich für jedes Jahr auf 100% und zeigt so die relativen Beiträge der einzelnen Energieträger innerhalb der beiden Hauptkategorien.
"""

In [11]:
# --- Panel-Dashboard erstellen (NUR SZENARIO 2 mit angepassten Titeln und Beschreibungen unter den Grafiken) ---
dashboard_f3_final = pn.Tabs(
    ("Abs. Aggregiert", pn.Column(
        pn.pane.Plotly(plot_f3_s2_absolut_aggregiert_tab()),
        pn.pane.Markdown(beschreibung_s2_absolut_aggregiert, width=800, styles={'background-color': 'white', 'color': 'black', 'padding': '10px'}) # Breite ggf. anpassen
    )),
    ("Abs. Detailliert", pn.Column(
        pn.pane.Plotly(plot_f3_s2_absolut_detail_tab()),
        pn.pane.Markdown(beschreibung_s2_absolut_detail, width=800, styles={'background-color': 'white', 'color': 'black', 'padding': '10px'})
    )),
    ("Anteil 100% Agg.", pn.Column(
        pn.pane.Plotly(plot_f3_s2_anteil_100prozent_aggregiert_tab()),
        pn.pane.Markdown(beschreibung_s2_anteil_100_aggregiert, width=800, styles={'background-color': 'white', 'color': 'black', 'padding': '10px'})
    )),
    ("Verhältnis", pn.Column(
        pn.pane.Plotly(plot_f3_s2_verhaeltnis_tab()),
        pn.pane.Markdown(beschreibung_s2_verhaeltnis, width=800, styles={'background-color': 'white', 'color': 'black', 'padding': '10px'})
    )),
    ("Anteil an Summe", pn.Column(
        pn.pane.Plotly(plot_f3_s2_anteil_summe_tab()),
        pn.pane.Markdown(beschreibung_s2_anteil_summe, width=800, styles={'background-color': 'white', 'color': 'black', 'padding': '10px'})
    )),
    ("Anteil 100% Detail", pn.Column(
        pn.pane.Plotly(plot_f3_s2_anteil_100prozent_detail_tab()),
        pn.pane.Markdown(beschreibung_s2_anteil_100_detail, width=800, styles={'background-color': 'white', 'color': 'black', 'padding': '10px'})
    )),
    dynamic=True
)

# --- Anzeige des Dashboards ---
dashboard_f3_final.servable(title="F3: Analyse Erneuerbare Inland vs. Fossile Importe (Szenario Müll)")

Tabs(dynamic=True, sizing_mode='stretch_width')
    [0] Column(sizing_mode='stretch_width')
        [0] Plotly(Figure, sizing_mode='stretch_width')
        [1] Markdown(str, styles={'background-color': 'whit...}, width=800)
    [1] Column(sizing_mode='stretch_width')
        [0] Plotly(Figure, sizing_mode='stretch_width')
        [1] Markdown(str, styles={'background-color': 'whit...}, width=800)
    [2] Column(sizing_mode='stretch_width')
        [0] Plotly(Figure, sizing_mode='stretch_width')
        [1] Markdown(str, styles={'background-color': 'whit...}, width=800)
    [3] Column(sizing_mode='stretch_width')
        [0] Plotly(Figure, sizing_mode='stretch_width')
        [1] Markdown(str, styles={'background-color': 'whit...}, width=800)
    [4] Column(sizing_mode='stretch_width')
        [0] Plotly(Figure, sizing_mode='stretch_width')
        [1] Markdown(str, styles={'background-color': 'whit...}, width=800)
    [5] Column(sizing_mode='stretch_width')
        [0] Plotly(Figure, sizing_mode='stretch_width')
        [1] Markdown(str, styles={'background-color': 'whit...}, width=800)

Analyse der Entwicklung

Die Analyse der aufbereiteten Daten offenbart eine dynamische, wenngleich von fossilen Energieträgern nach wie vor dominierte Entwicklung im Verhältnis zwischen inländisch produzierter erneuerbarer Energie und importierter fossiler Energie.

Die Betrachtung der absoluten Energiemengen (siehe Grafik "Absolute Mengen (aggregiert): Inl. Erneuerbare vs. Fossile Importe") verdeutlicht, dass die Importe fossiler Energieträger über den gesamten Zeitraum hinweg die mengenmäßig stärkere Komponente darstellten. Diese Importe unterlagen zwar Schwankungen, bedingt durch ökonomische Zyklen und Preisentwicklungen, verblieben jedoch auf einem durchgehend hohen Niveau, wenngleich in den allerjüngsten Jahren ein tendenzieller Rückgang zu beobachten ist. Parallel dazu zeigt die inländische Produktion erneuerbarer Energien einen kontinuierlichen, wenngleich im Vergleich zu den fossilen Importen in absoluten Zahlen moderateren Anstieg. Die detaillierte Aufschlüsselung (Grafik "Absolute Mengen (detailliert): Inl. Erneuerbare vs. Fossile Importe") zeigt, dass die Wasserkraft traditionell das Rückgrat der inländischen erneuerbaren Produktion bildet. Jedoch gewinnen "Uebrige erneuerbare Energien" – ein Sammelbegriff, der unter anderem Solarenergie und Windkraft umfasst – sowie der energetisch genutzte Anteil aus "Müll und Industrieabfällen" in den letzten Jahren zunehmend an Bedeutung und tragen maßgeblich zum Wachstum der erneuerbaren Inlandproduktion bei. Aufseiten der fossilen Importe bleiben Erdölprodukte und Erdgas die dominierenden Energieträger.

Die anteilige Entwicklung, visualisiert im 100%-Flächendiagramm (Grafik "Anteilige Entwicklung (aggregiert, 100% Ansicht)"), unterstreicht diese graduelle Verschiebung. Der prozentuale Anteil der inländischen erneuerbaren Energien an der Summe aus (inländischen Erneuerbaren + fossilen Importen) hat im Betrachtungszeitraum von 1980 bis 2023 langsam, aber stetig zugenommen. Während dieser Anteil zu Beginn des Beobachtungszeitraums noch deutlich geringer war, ist er bis zum Jahr 2023 merklich angestiegen. Dies illustriert eine langsame, aber fortschreitende Veränderung im betrachteten Energiemix, auch wenn die fossilen Importe weiterhin den größten Anteil ausmachen. Die detaillierte 100%-Ansicht (Grafik "Anteilige Entwicklung (detailliert, 100% Ansicht)") gibt zudem Aufschluss über die internen Verschiebungen innerhalb der einzelnen Energieträgergruppen.

Besonders aussagekräftig ist die Entwicklung des direkten Verhältnisses der inländischen erneuerbaren Produktion zu den fossilen Importen (Grafik "Verhältnis Inl. Erneuerbare zu Fossilen Importen"). Dieser Quotient, der 1980 noch auf einem vergleichsweise niedrigen Niveau lag, zeigt über die Jahrzehnte einen klaren Aufwärtstrend. Obwohl der Wert durchgehend unter der Marke von 1 verblieb – was bedeutet, dass die Menge der importierten fossilen Energien stets höher war als die der im Inland produzierten erneuerbaren Energien (bezogen auf die definierten Kategorien) – signalisiert der Anstieg eine relative Stärkung der erneuerbaren Inlandsproduktion. Die Grafik "Anteil Inl. Erneuerbare an Summe (Inl. Ern. + Foss. Imp.)" bestätigt diesen Trend, indem sie den wachsenden Prozentsatz der heimischen Erneuerbaren an der kombinierten Gesamtmenge visualisiert.

Diese beobachteten Entwicklungen sind maßgeblich durch eine Reihe von Einflussfaktoren und energiepolitischen Kontextualisierungen geprägt. Technologische Fortschritte und Kostensenkungen, insbesondere bei Photovoltaik und Windkraft, haben den Ausbau erneuerbarer Energien begünstigt. Gleichzeitig beeinflussen schwankende Weltmarktpreise für Öl und Gas die Importmengen. Ein entscheidender Rahmenfaktor ist die Schweizer Energiepolitik, insbesondere die Energiestrategie 2050. Diese zielt auf einen schrittweisen Ausstieg aus der Kernenergie, eine massive Reduktion des Verbrauchs fossiler Energien und eine damit einhergehende deutliche Steigerung des Anteils erneuerbarer Energien sowie der Energieeffizienz ab. Politische Förderinstrumente wie die Einspeisevergütung, Einmalvergütungen für Photovoltaikanlagen und verschiedene kantonale Programme haben den Ausbau der erneuerbaren Energien zusätzlich stimuliert. Darüber hinaus spielen ein wachsendes gesellschaftliches Bewusstsein für den Klimawandel und das Bestreben nach einer höheren Versorgungssicherheit eine wichtige Rolle bei der Gestaltung der Energiezukunft. Internationale Abkommen, wie das Pariser Klimaabkommen, setzen zudem verbindliche Reduktionsziele für Treibhausgasemissionen, was den Druck zur Abkehr von fossilen Energieträgern erhöht.

Bewertung im Kontext der Energiestrategie 2050 und Handlungsempfehlungen

Die Analyse der Energiestatistik von 1980 bis 2023 zeigt eine schrittweise, aber deutliche Veränderung im Verhältnis zwischen inländisch produzierter erneuerbarer Energie und den Importen fossiler Energieträger. Der Anteil der Erneuerbaren hat zugenommen, was im Einklang mit den übergeordneten Zielen der Energiestrategie 2050 steht. Diese Strategie verfolgt das Kernziel, die Schweiz langfristig mit ausreichend, breit gefächerter, sicherer, wirtschaftlicher und umweltverträglicher Energie zu versorgen und gleichzeitig die energiebedingten CO2-Emissionen drastisch zu reduzieren.

Aus der fiktiven Perspektive von Mitarbeitern des Bundesamtes für Energie (BFE), deren Aufgabe es ist, die Umsetzung der Energiestrategie zu begleiten und zu fördern, lassen die Ergebnisse folgende Bewertungen und Handlungsempfehlungen zur Beschleunigung der Transformation zu:

- Ausbaupfad der erneuerbaren Energien konsequent weiterverfolgen und intensivieren: Der beobachtete Zuwachs bei den "Uebrigen erneuerbaren Energien" und dem energetisch genutzten Müllanteil ist positiv, muss aber signifikant beschleunigt werden, um die ambitionierten Ziele der Energiestrategie zu erreichen und die nach wie vor hohe Dominanz fossiler Importe zu brechen.
 - BFE-Perspektive: Es gilt, die Förderinstrumente (z.B. Investitionsbeiträge, wettbewerbliche Ausschreibungen) kontinuierlich zu evaluieren und zu optimieren, um maximale Wirksamkeit zu erzielen. Administrative Hürden für den Ausbau, insbesondere bei Windkraft und Groß-PV-Anlagen, müssen weiter abgebaut und Genehmigungsverfahren beschleunigt werden. Die Sensibilisierung und Information der Bevölkerung über die Notwendigkeit und die Chancen des Ausbaus sind dabei essenziell.

- Fokus auf Sektorkopplung und Effizienz zur Reduktion fossiler Importe: Die Reduktion der fossilen Importe hängt nicht nur vom Zubau Erneuerbarer ab, sondern maßgeblich auch von Effizienzsteigerungen und der Elektrifizierung bzw. dem Einsatz erneuerbarer Energien in den Verbrauchssektoren Wärme und Mobilität.
 - BFE-Perspektive: Programme zur Gebäudesanierung und zum Ersatz fossiler Heizsysteme durch Wärmepumpen oder den Anschluss an Wärmenetze müssen weiter gestärkt werden. Die Förderung der Elektromobilität und erneuerbarer Treibstoffe ist ebenso zentral. Informationskampagnen können das Bewusstsein für Einsparpotenziale schärfen.

- Stärkung der Versorgungssicherheit durch Diversifizierung und Speicher: Eine geringere Abhängigkeit von fossilen Importen erhöht die Versorgungssicherheit. Dies erfordert nicht nur den Ausbau der heimischen Produktion, sondern auch Investitionen in Speichertechnologien und intelligente Netze, um die Volatilität von Solar- und Windenergie auszugleichen.
 - BFE-Perspektive: Die Rahmenbedingungen für saisonale Speicher (z.B. Pumpspeicher, Power-to-Gas) müssen verbessert und der Ausbau intelligenter Stromnetze (Smart Grids) vorangetrieben werden. Die Rolle der Schweiz im europäischen Stromverbund muss im Kontext der Versorgungssicherheit aktiv gestaltet werden.

- Transparente Erfolgsmessung und adaptive Strategieanpassung: Die Entwicklung des Verhältnisses von inländischen Erneuerbaren zu fossilen Importen ist ein wichtiger Indikator. Ein kontinuierliches Monitoring, gestützt auf aussagekräftige Visualisierungen, ermöglicht es, die Wirksamkeit der Maßnahmen zu überprüfen und die Strategie bei Bedarf adaptiv anzupassen.
 - BFE-Perspektive: Die regelmäßige Publikation von Fortschrittsberichten, wie der Schweizerischen Gesamtenergiestatistik, und die verständliche Aufbereitung der Daten für Entscheidungsträger und die Öffentlichkeit sind unerlässlich. Die Entwicklung und Nutzung interaktiver Visualisierungstools, wie sie im Rahmen dieser Case Study konzipiert wurden, können hierbei einen wertvollen Beitrag leisten.

Fazit

Die Analyse der Schweizerischen Gesamtenergiestatistik von 1980 bis 2023 zeigt eine positive, wenn auch langsame Verschiebung zugunsten der inländischen erneuerbaren Energien im Vergleich zu den Importen fossiler Energieträger. Das Verhältnis hat sich verbessert, und der prozentuale Beitrag der Erneuerbaren (unter Einbezug von 50% des energetisch genutzten Mülls) zur Summe der beiden betrachteten Energieblöcke ist gestiegen. Diese Entwicklung ist ein Indiz dafür, dass die energiepolitischen Weichenstellungen erste Früchte tragen.

Dennoch verdeutlicht die Analyse auch die weiterhin erhebliche Abhängigkeit der Schweiz von fossilen Energieimporten. Um die Ziele der Energiestrategie 2050 – insbesondere eine drastische Reduktion der CO2-Emissionen und eine Stärkung der einheimischen, erneuerbaren Energieversorgung – zu erreichen, ist eine deutliche Beschleunigung des Transformationsprozesses unabdingbar. Die dargestellten Handlungsempfehlungen skizzieren aus Sicht des BFE mögliche Wege, um den Ausbau erneuerbarer Energien zu forcieren, die Effizienz zu steigern und somit die Abhängigkeit von fossilen Energieträgern nachhaltig zu reduzieren. Die kontinuierliche und transparente Beobachtung der relevanten Kennzahlen bleibt dabei ein zentrales Element, um den Fortschritt zu messen und die Strategie zielführend weiterzuentwickeln.